In [1]:
# windows function
from pyspark.sql import SparkSession
import pyspark.sql.functions as F 
from pyspark.sql import Window
import pyspark.sql.types as T

In [2]:
spark=SparkSession.builder.appName("Windows Function").getOrCreate()

22/10/19 11:35:41 WARN Utils: Your hostname, HP-G62 resolves to a loopback address: 127.0.1.1; using 192.168.18.113 instead (on interface enp3s0)
22/10/19 11:35:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/19 11:35:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/19 11:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/19 11:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/19 11:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/19 11:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/19 11:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/10/19 11:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/10/19 11:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/10/19 11:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.


In [3]:
employee_df=spark.read.csv("employees.csv",inferSchema=True,header=True)
employee_df= employee_df.select(["FIRST_NAME","SALARY","DEPARTMENT_ID"])

In [4]:
employee_df.show()

+----------+------+-------------+
|FIRST_NAME|SALARY|DEPARTMENT_ID|
+----------+------+-------------+
|    Donald|  2600|           50|
|   Douglas|  2600|           50|
|  Jennifer|  4400|           10|
|   Michael| 13000|           20|
|       Pat|  6000|           20|
|     Susan|  6500|           40|
|   Hermann| 10000|           70|
|   Shelley| 12008|          110|
|   William|  8300|          110|
|    Steven| 24000|           90|
|     Neena| 17000|           90|
|       Lex| 17000|           90|
| Alexander|  9000|           60|
|     Bruce|  6000|           60|
|     David|  4800|           60|
|     Valli|  4800|           60|
|     Diana|  4200|           60|
|     Nancy| 12008|          100|
|    Daniel|  9000|          100|
|      John|  8200|          100|
+----------+------+-------------+
only showing top 20 rows



In [5]:
employee_df.count()

50

In [6]:
type(employee_df)

pyspark.sql.dataframe.DataFrame

In [7]:
#  making windows spec for partition by department id
windowSpec=Window.partitionBy("DEPARTMENT_ID")

In [8]:
#  finding the salary list, average salary and total salary of each department using windows function 
win_emp_df=employee_df.withColumn("List Salary",F.collect_list(F.col("SALARY")).over(windowSpec))\
    .withColumn("Average Salary",F.avg(F.col("SALARY")).over(windowSpec))\
    .withColumn("Total Salary",F.sum(F.col("SALARY")).over(windowSpec))
win_emp_df.show()

+----------+------+-------------+--------------------+------------------+------------+
|FIRST_NAME|SALARY|DEPARTMENT_ID|         List Salary|    Average Salary|Total Salary|
+----------+------+-------------+--------------------+------------------+------------+
|  Jennifer|  4400|           10|              [4400]|            4400.0|        4400|
|   Michael| 13000|           20|       [13000, 6000]|            9500.0|       19000|
|       Pat|  6000|           20|       [13000, 6000]|            9500.0|       19000|
|       Den| 11000|           30|[11000, 3100, 290...|            4150.0|       24900|
| Alexander|  3100|           30|[11000, 3100, 290...|            4150.0|       24900|
|    Shelli|  2900|           30|[11000, 3100, 290...|            4150.0|       24900|
|     Sigal|  2800|           30|[11000, 3100, 290...|            4150.0|       24900|
|       Guy|  2600|           30|[11000, 3100, 290...|            4150.0|       24900|
|     Karen|  2500|           30|[11000, 31

In [9]:
win_emp_df.count()

50

In [10]:
#  making windows spec for partition by department id
windowSpec=Window.partitionBy("DEPARTMENT_ID").orderBy(F.asc("SALARY"))

In [11]:
#  finding the salary list, min salary,average salary, max salary and total salary of each department using windows function 
win_emp_df=employee_df.withColumn("List Salary",F.collect_list(F.col("SALARY")).over(windowSpec))\
    .withColumn("Min Salary",F.min(F.col("SALARY")).over(windowSpec))\
    .withColumn("Average Salary",F.avg(F.col("SALARY")).over(windowSpec))\
    .withColumn("Max Salary",F.max(F.col("SALARY")).over(windowSpec))\
    .withColumn("Total Salary",F.sum(F.col("SALARY")).over(windowSpec))\
    .withColumn("Salary Count",F.count(F.col("SALARY")).over(windowSpec))
    
win_emp_df.show()

+----------+------+-------------+--------------------+----------+------------------+----------+------------+------------+
|FIRST_NAME|SALARY|DEPARTMENT_ID|         List Salary|Min Salary|    Average Salary|Max Salary|Total Salary|Salary Count|
+----------+------+-------------+--------------------+----------+------------------+----------+------------+------------+
|  Jennifer|  4400|           10|              [4400]|      4400|            4400.0|      4400|        4400|           1|
|       Pat|  6000|           20|              [6000]|      6000|            6000.0|      6000|        6000|           1|
|   Michael| 13000|           20|       [6000, 13000]|      6000|            9500.0|     13000|       19000|           2|
|     Karen|  2500|           30|              [2500]|      2500|            2500.0|      2500|        2500|           1|
|       Guy|  2600|           30|        [2500, 2600]|      2500|            2550.0|      2600|        5100|           2|
|     Sigal|  2800|     

In [12]:
win_emp_df.collect()[2]["List Salary"]

[6000, 13000]

In [13]:
win_emp_df.collect()[13]["List Salary"]

[2100, 2200, 2200, 2400, 2400]

In [14]:
win_emp_df.collect()[13]["Max Salary"]

2400

In [15]:
win_emp_df.collect()[10]["Min Salary"]

2100

In [16]:
win_emp_df.select(["DEPARTMENT_ID","List Salary","Min Salary","Average Salary","Max Salary","Total Salary"]).distinct().show()

+-------------+--------------------+----------+------------------+----------+------------+
|DEPARTMENT_ID|         List Salary|Min Salary|    Average Salary|Max Salary|Total Salary|
+-------------+--------------------+----------+------------------+----------+------------+
|           10|              [4400]|      4400|            4400.0|      4400|        4400|
|           20|              [6000]|      6000|            6000.0|      6000|        6000|
|           20|       [6000, 13000]|      6000|            9500.0|     13000|       19000|
|           30|              [2500]|      2500|            2500.0|      2500|        2500|
|           30|        [2500, 2600]|      2500|            2550.0|      2600|        5100|
|           30|  [2500, 2600, 2800]|      2500|2633.3333333333335|      2800|        7900|
|           30|[2500, 2600, 2800...|      2500|            2700.0|      2900|       10800|
|           30|[2500, 2600, 2800...|      2500|            2780.0|      3100|       13900|

In [17]:
#  it is equivalent to the result from the distinct result after windows function
grp_emp_df=employee_df.groupBy("DEPARTMENT_ID").agg(
    F.expr("collect_list(SALARY)").alias("List Salary"),
    F.expr("min(SALARY)").alias("Min Salary"),
    F.expr("avg(SALARY)").alias("Average Salary"),
    F.expr("max(SALARY)").alias("Max Salary"),
    F.expr("sum(SALARY)").alias("Total Salary"),
    F.expr("count(SALARY)").alias("Salary Count"))

grp_emp_df.show()

+-------------+--------------------+----------+------------------+----------+------------+------------+
|DEPARTMENT_ID|         List Salary|Min Salary|    Average Salary|Max Salary|Total Salary|Salary Count|
+-------------+--------------------+----------+------------------+----------+------------+------------+
|           20|       [13000, 6000]|      6000|            9500.0|     13000|       19000|           2|
|           40|              [6500]|      6500|            6500.0|      6500|        6500|           1|
|          100|[12008, 9000, 820...|      6900| 8601.333333333334|     12008|       51608|           6|
|           10|              [4400]|      4400|            4400.0|      4400|        4400|           1|
|           50|[2600, 2600, 8000...|      2100|3721.7391304347825|      8200|       85600|          23|
|           70|             [10000]|     10000|           10000.0|     10000|       10000|           1|
|           90|[24000, 17000, 17...|     17000|19333.33333333333

In [18]:
grp_emp_df.select(["DEPARTMENT_ID","Min Salary","Max Salary"]).show()

+-------------+----------+----------+
|DEPARTMENT_ID|Min Salary|Max Salary|
+-------------+----------+----------+
|           20|      6000|     13000|
|           40|      6500|      6500|
|          100|      6900|     12008|
|           10|      4400|      4400|
|           50|      2100|      8200|
|           70|     10000|     10000|
|           90|     17000|     24000|
|           60|      4200|      9000|
|          110|      8300|     12008|
|           30|      2500|     11000|
+-------------+----------+----------+



In [20]:
grp_emp_df.show()

+-------------+--------------------+----------+------------------+----------+------------+------------+
|DEPARTMENT_ID|         List Salary|Min Salary|    Average Salary|Max Salary|Total Salary|Salary Count|
+-------------+--------------------+----------+------------------+----------+------------+------------+
|           20|       [13000, 6000]|      6000|            9500.0|     13000|       19000|           2|
|           40|              [6500]|      6500|            6500.0|      6500|        6500|           1|
|          100|[12008, 9000, 820...|      6900| 8601.333333333334|     12008|       51608|           6|
|           10|              [4400]|      4400|            4400.0|      4400|        4400|           1|
|           50|[2600, 2600, 8000...|      2100|3721.7391304347825|      8200|       85600|          23|
|           70|             [10000]|     10000|           10000.0|     10000|       10000|           1|
|           90|[24000, 17000, 17...|     17000|19333.33333333333

In [21]:
win_emp_df.show()

+----------+------+-------------+--------------------+----------+------------------+----------+------------+------------+
|FIRST_NAME|SALARY|DEPARTMENT_ID|         List Salary|Min Salary|    Average Salary|Max Salary|Total Salary|Salary Count|
+----------+------+-------------+--------------------+----------+------------------+----------+------------+------------+
|  Jennifer|  4400|           10|              [4400]|      4400|            4400.0|      4400|        4400|           1|
|       Pat|  6000|           20|              [6000]|      6000|            6000.0|      6000|        6000|           1|
|   Michael| 13000|           20|       [6000, 13000]|      6000|            9500.0|     13000|       19000|           2|
|     Karen|  2500|           30|              [2500]|      2500|            2500.0|      2500|        2500|           1|
|       Guy|  2600|           30|        [2500, 2600]|      2500|            2550.0|      2600|        5100|           2|
|     Sigal|  2800|     